In [ ]:
from IPython.display import display, Math
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import scipy as scipy    

# Eigenvectors of the stiffness matrix in limit state problems
[Dominik Kern](https://orcid.org/0000-0002-1958-2982),
[Chiara Hergl](https://orcid.org/0000-0002-4016-9113),
[Thomas Nagel](https://orcid.org/0000-0001-8459-4616)

**TODO** enter final title!
# Minimal Example of Beam Buckling with Axial-transversal Coupling
With this formulation we obtain a common FEM-like equation $\mathbf{C}\mathbf{u} = \mathbf{f}$ with $\mathbf{C}=\mathbf{C}({\mathbf{u}})$.

![Euler buckling case 2](euler_buckling.svg)

Non-dimensional potential energy for coupled beam bending (transversal displacement $w$, axial displacement $u$) with axial force $F$ and stabilizing spring (stiffness $c$)    
$\Pi = \frac{1}{2}\int_0^1 w''(x)^2 + G \left(u' + \frac{1}{2} w'^2\right)^2 \mathrm{d}x +\frac{1}{2}cw(X)^2 + F u(1)$,

and following relations to the dimensional variables 
- $G = \dfrac{A l^2}{I}$ squared slenderness ratio,
- $c = \dfrac{c_\mathrm{dim}l}{F_\mathrm{ref}}$,
- $F = \dfrac{F_\mathrm{dim}}{F_\mathrm{ref}}$,
with $F_\mathrm{ref} = \dfrac{EI}{l^2}$, note that first buckling load of this case is $F_\mathrm{crit} = \pi^2 F_\mathrm{ref}$.

Ritz' method with discretization $w(x)\approx \sum\limits_{k=1}^{K} \sin\bigl(k\pi x \bigr)$ and $u\approx \hat{u}x$, here $K=2$.

## Symbolic calculations

Apply Ritz discretization (3 DoF: 1 axial and 2 transversal) and evaluate first variation of potential energy

In [ ]:
x, X, k, c, F, W1, W2, dW1, dW2, U1, dU1, G  = sp.symbols('x X k c F W1 W2 dW1 dW2 U1 dU1 G')

In [ ]:
w = sp.sin(k*sp.pi*x) 

W = W1*w.subs(k,1) + W2*w.subs(k,2)
Wx = W.diff(x)
Wxx = Wx.diff(x)

dW = dW1*w.subs(k,1) + dW2*w.subs(k,2)
dWx = dW.diff(x)
dWxx = dWx.diff(x)

U = U1*x
Ux = U.diff(x)

dU = dU1*x
dUx = dU.diff(x)

dPi = sp.integrate(Wxx*dWxx + G*(Ux+(1/2)*Wx*Wx)*(dUx+Wx*dWx), (x,0,1) ) + c*W.subs(x,X)*dW.subs(x,X) + F*U.subs(x,1)
#display(dPi)

Generalized forces, conjugated with $w_1$, $w_2$ and $u_1$, respectively.

In [ ]:
Q1 = sp.simplify(dPi.diff(W1))
display(Math(f'Q_1 = {sp.latex(Q1)}'))
Q2 = sp.simplify(dPi.diff(W2))
display(Math(f'Q_2 = {sp.latex(Q2)}'))
Q3 = sp.simplify(dPi.diff(U1))
display(Math(f'Q_3 = {sp.latex(Q3)}'))

Stiffness matrix (symmetric, remember $\sin(2x)=2\sin(x)\cos(x)$)

In [ ]:
C11 = sp.expand(Q1).coeff(dW1, 1)
C12 = sp.expand(Q1).coeff(dW2, 1)
C13 = sp.expand(Q1).coeff(dU1, 1)

C21 = sp.expand(Q2).coeff(dW1, 1)
C22 = sp.expand(Q2).coeff(dW2, 1)
C23 = sp.expand(Q2).coeff(dU1, 1)

C31 = sp.expand(Q3).coeff(dW1, 1)
C32 = sp.expand(Q3).coeff(dW2, 1)
C33 = sp.expand(Q3).coeff(dU1, 1)

C = sp.Matrix(((C11, C12, C13), (C21, C22, C23), (C31, C32, C33)))
display(Math(f'C = {sp.latex(C)}'))

Stiffness matrix before buckling, i.e. without bending deformation $w(x)=0$.

In [ ]:
nDoF = 3
Wzero = 0
Czero = C.subs([(W1, Wzero), (W2, Wzero)])
display(Math(f'C_0 = {sp.latex(Czero)}'))

Buckling occurs when the determinant of the stiffness matrix vanishes.

In [ ]:
char_eq = sp.simplify(sp.Determinant(C).doit())
buckling_state = sp.solve(char_eq, U1)
print(str(len(buckling_state)) + " buckling states (as many as transversal DoF)")
U_1 = sp.simplify(buckling_state[0])
U_2 = sp.simplify(buckling_state[1])
Uzero_1 = U_1.subs([(W1, Wzero), (W2, Wzero)])
Uzero_2 = U_2.subs([(W1, Wzero), (W2, Wzero)])

## Numerical evaluations

common value for the slenderness ratio and a wide range of spring stiffnesses

In [ ]:
N = 100
c_grid = np.linspace(0, 300,N)   # spring stiffness
Gnum = 100**2   # squarred slenderness ratio
num_zero = 1e-10

F_lower_sym = np.zeros((N,1))
F_higher_sym = np.zeros((N,1))

F_lower_asym = np.zeros((N,1))
F_higher_asym = np.zeros((N,1))
gev_lower = np.zeros((N,3))
gev_higher = np.zeros((N,3))
sev_lower = np.zeros((N,3))
sev_higher = np.zeros((N,3))

### stabilizing spring in the middle ($X=\frac{1}{2}$) _symmetric case_

In [ ]:
Xsym = 0.5

Set up generalized eigenvalue problem $\mathbf{A}\mathbf{w} = x\mathbf{B}\mathbf{w}$ and compare with standard eigenvectors at $U=U_\mathrm{crit}$


In [ ]:
for k,cnum in enumerate(c_grid):
    Czero_sym = Czero.subs([(X,Xsym),(c,cnum),(G,Gnum)])
    C33num = Czero_sym.row(2).col(2)[0]  # strictly speaking, here we'd need matrix-vector multiplication, but we know matrix structure
    
    # C = A + B*U1, since we specifically know the matrix structure for this example problem   
    Azero_sym = sp.Matrix([[( Czero_sym[i,j].coeff(U1,0).evalf()) for i in range(nDoF)] for j in range(nDoF)])
    Bzero_sym = sp.Matrix([[(-Czero_sym[i,j].coeff(U1,1).evalf()) for i in range(nDoF)] for j in range(nDoF)])
    Azero_num = np.array(Azero_sym).astype(np.float64)
    Bzero_num = np.array(Bzero_sym).astype(np.float64)
    eigvals, eigvecs = scipy.linalg.eig(Azero_num, b=Bzero_num)
    
    # sort by eigenvalues
    if np.abs(eigvals[2]) < np.abs(eigvals[0]) or np.abs(eigvals[2]) < np.abs(eigvals[1]):
        print("Warning, for k={} assumption of third eigenvalue being infinite is not fulfilled!".format(k))
    
    if np.abs(eigvals[0]) < np.abs(eigvals[1]):
        F_lower_sym[k]  = eigvals[0].real*C33num.evalf()
        F_higher_sym[k] = eigvals[1].real*C33num.evalf()
    else:
        F_lower_sym[k]  = eigvals[1].real*C33num.evalf()
        F_higher_sym[k] = eigvals[0].real*C33num.evalf()

    #F_inf_sym[k] = eigvals[2].real*C33num   # difficult to plot

Plot buckling load for a range of spring stiffnesses

In [ ]:
plt.plot(c_grid,np.abs(F_lower_sym),'b', c_grid,np.abs(F_higher_sym),'r')
plt.xlabel("stiffness");
plt.ylabel("buckling load");
plt.legend(["|F_1|", "|F_2|"]);

Note that the horizontal line, independent on color, corresponds to the second mode shape and the sloped line to the first mode shape. The physics behind this plot are that, the spring only stabilizes against first mode buckling and is ineffective for the second mode (node point). There is one value of the spring stiffness, when both buckling loads are equal.

### stabilizing spring somewhere ($X\ne\frac{1}{2}$) _asymmetric case_
By _asymmetry_ we refer to the vertical direction, as in horizontal direction symmetry is preserved, i.e. no difference in bending to the right or to the left.

In [ ]:
Xasym = 0.55   

In [ ]:
for k,cnum in enumerate(c_grid):
    Czero_asym = Czero.subs([(X,Xasym),(c,cnum),(G,Gnum)])
    C33num = Czero_asym.row(2).col(2)[0]
    
    Azero_asym = sp.Matrix([[( Czero_asym[i,j].coeff(U1,0).evalf()) for i in range(nDoF)] for j in range(nDoF)])
    Bzero_asym = sp.Matrix([[(-Czero_asym[i,j].coeff(U1,1).evalf()) for i in range(nDoF)] for j in range(nDoF)])
    Azero_anum = np.array(Azero_asym).astype(np.float64)
    Bzero_anum = np.array(Bzero_asym).astype(np.float64)
    eigvals, eigvecs = scipy.linalg.eig(Azero_anum, b=Bzero_anum)   # generalized Eigenvalue problem  
    
    # sort by eigenvalues
    if np.abs(eigvals[2]) < np.abs(eigvals[0]) or np.abs(eigvals[2]) < np.abs(eigvals[1]):
        print("Warning, for k={} assumption of third eigenvalue being infinite is not fulfilled!".format(k))
        
    if np.abs(eigvals[0]) < np.abs(eigvals[1]): 
        U1lower  = eigvals[0]
        F_lower_asym[k] = eigvals[0].real*C33num.evalf()
        gev_lower[k,:] = eigvecs[:,0]
        U1higher = eigvals[1]
        F_higher_asym[k] = eigvals[1].real*C33num.evalf()
        gev_higher[k,:] = eigvecs[:,1]
    
    else:
        U1lower  = eigvals[1]
        F_lower_asym[k] = eigvals[1].real*C33num.evalf()
        gev_lower[k,:] = eigvecs[:,1]
        U1higher = eigvals[0]
        F_higher_asym[k] = eigvals[0].real*C33num.evalf()
        gev_higher[k,:] = eigvecs[:,0]

    # compute eigenvector (zero eigenvalue) at lower buckling load
    Czero_lower =  np.matrix(Czero_asym.subs(U1, U1lower).evalf()).astype(np.float64)
    standard_eigvals, standard_eigvecs = scipy.linalg.eigh(Czero_lower)
    # pick eigenvector of eigenvalue = 0
    i_min = np.argmin(np.abs(standard_eigvals))
    if np.abs(standard_eigvals[i_min]) > num_zero:
        print("Warning, minimal eigenvalue (SEVP) at buckling must be zero!")
    else:
        sev_lower[k,:] = standard_eigvecs[:,i_min]

    # compute eigenvector (zero eigenvalue) at higher buckling load
    Czero_higher =  np.matrix(Czero_asym.subs(U1, U1higher).evalf()).astype(np.float64)
    standard_eigvals, standard_eigvecs = scipy.linalg.eigh(Czero_higher)
    # pick eigenvector of eigenvalue = 0
    i_min = np.argmin(np.abs(standard_eigvals))
    if np.abs(standard_eigvals[i_min]) > num_zero:
        print("Warning, minimal eigenvalue (SEVP) at buckling must be zero!")
    else:
        sev_higher[k,:] = standard_eigvecs[:,i_min]
        
    #F_inf_asym[k] = eigvals[2].real*C33num   # difficult to plot

In [ ]:
plt.plot(c_grid,np.abs(F_lower_asym),'b', c_grid,np.abs(F_higher_asym),'r')
plt.xlabel("stiffness");
plt.ylabel("buckling load");
plt.legend(["|F_1|", "|F_2|"]);

There is no common value of two buckling modes for any spring stiffness.

#### Eigenvectors

##### Generalized EVP

In [ ]:
plt.plot(c_grid, np.abs(gev_lower[:,0]),'b', c_grid, np.abs(gev_lower[:,1]),'b--', c_grid, np.abs(gev_lower[:,2]),'b:');
#plt.plot(c_grid, np.abs(ev1[:,0])**2 + np.abs(ev1[:,1])**2 + np.abs(ev1[:,2])**2, 'gray');
plt.xlabel('$c$');
plt.legend(['$|w_1|$', '$|w_2|$', '$|u_1|$']);

##### Standard EVP

In [ ]:
plt.plot(c_grid, np.abs(sev_lower[:,0]),'c', c_grid, np.abs(sev_lower[:,1]),'c--', c_grid, np.abs(sev_lower[:,2]),'c:');
#plt.plot(c_grid, np.abs(ev_l1[:,0])**2 + np.abs(ev_l1[:,1])**2 + np.abs(ev_l1[:,2])**2, 'gray');
plt.xlabel('$c$');
plt.legend(['$|w_1|$', '$|w_2|$', '$|u_1|$']);

For the lower buckling load there is a transition from the first mode shape to the second. Both eigenvectors, from the generalized and from the standard EVP, are identical as expected. 

##### Generalized EVP

In [ ]:
plt.plot(c_grid, np.abs(gev_higher[:,0]),'r', c_grid, np.abs(gev_higher[:,1]),'r--', c_grid, np.abs(gev_higher[:,2]),'r:');
#plt.plot(c_grid, np.abs(ev2[:,0])**2 + np.abs(ev2[:,1])**2 + np.abs(ev2[:,2])**2, 'gray');
plt.xlabel('$c$');
plt.legend(['$|w_1|$', '$|w_2|$', '$|u_1|$']);

##### Standard EVP

In [ ]:
plt.plot(c_grid, np.abs(sev_higher[:,0]),'m', c_grid, np.abs(sev_higher[:,1]),'m--', c_grid, np.abs(sev_higher[:,2]),'m:');
#plt.plot(c_grid, np.abs(ev_h1[:,0])**2 + np.abs(ev_h1[:,1])**2 + np.abs(ev_h1[:,2])**2, 'gray');
plt.xlabel('$c$');
plt.legend(['$|w_1|$', '$|w_2|$', '$|u_1|$']);

For the higher buckling load it is the opposite transition than for the lower buckling load. Again, both eigenvectors match.

#### Plots of the buckling shapes for different values of the spring stiffness

In [ ]:
x_grid = np.linspace(0, 1) 
index_c = [1, 48, 99]    # low, intermediate and high spring stiffness (manual selection)
for i_c in index_c:
    w_buckling_lower_load  = lambda x: gev_lower[i_c, 0] * np.sin(np.pi*x) + gev_lower[i_c, 1] * np.sin(2*np.pi*x) 
    w_buckling_higher_load = lambda x: gev_higher[i_c, 0] * np.sin(np.pi*x) + gev_higher[i_c, 1] * np.sin(2*np.pi*x) 
    plt.plot(x_grid, w_buckling_lower_load(x_grid),'b')
    plt.plot(x_grid, w_buckling_higher_load(x_grid),'r')
    plt.xlabel('x')
    plt.ylabel('w')
    plt.ylim([-1.25,1.25])
    plt.legend(['F_1', 'F_2'])
    plt.title('c={:.2f}'.format(c_grid[i_c]))
    plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1, Xasym], ['0', '0.2', '0.4', '0.6', '0.8', '1', 'X'])
    plt.show(); 